## pinecone
- 크로마와 같은 벡터 데이터베이스
- 크로마는 인메모리인 반면 pinecone은 클라우드를 활용한다
https://python.langchain.com/docs/integrations/vectorstores/pinecone/
https://app.pinecone.io/organizations/-OclQ8NG9cmThk3mA5TA/projects/7e470be4-fdc8-4ba1-93f0-3aade0b3f98b/indexes

In [1]:
%pip install --upgrade pip
%pip install -qU langchain langchain-pinecone langchain-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Pinecone as PineconeVectorStore
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


def LLM(user_message):
    # 1️⃣ 환경변수 로드 (.env에서 API 키 불러옴)
    load_dotenv()

    # 2️⃣ 임베딩 모델
    embedding = OpenAIEmbeddings(model="text-embedding-3-large")

    # 3️⃣ Pinecone 기존 인덱스 불러오기
    index_name = "tax-index"
    database = PineconeVectorStore.from_existing_index(
        index_name=index_name,
        embedding=embedding
    )

    # 4️⃣ LLM 초기화
    llm = ChatOpenAI(model="gpt-4o")

    # 5️⃣ 사용자 단어 → 세무 용어 변환용 사전
    dictionary = {
        "직장인": "거주자",
        "근로자": "거주자",
        "월급": "근로소득",
        "세금": "소득세"
    }

    # 6️⃣ 질문을 변환하기 위한 사전 체인
    prompt = ChatPromptTemplate.from_template("""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요.

    사전: {dictionary}
    질문: {question}
    """)

    dictionary_chain = prompt | llm | StrOutputParser()

    # 7️⃣ 변환된 질문 생성
    new_query = dictionary_chain.invoke({
        "dictionary": dictionary,
        "question": user_message
    })

    # 8️⃣ 벡터DB에서 관련 문서 검색
    retrieved_docs = database.similarity_search(new_query, k=3)

    # 9️⃣ 문서 내용 합치기
    context_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

    # 🔟 최종 RAG 프롬프트 구성
    rag_prompt_template = PromptTemplate(
        template="""
        You are a helpful assistant that can answer questions about Korean tax law.
        Use the following context to answer accurately and concisely in Korean.

        Context:
        {context}

        Question:
        {question}

        Answer:
        """,
        input_variables=["context", "question"]
    )

    # 11️⃣ 체인 연결
    rag_chain = rag_prompt_template | llm | StrOutputParser()

    # 12️⃣ 답변 생성
    ai_message = rag_chain.invoke({
        "context": context_text,
        "question": new_query
    })

    return ai_message


In [ ]:
import os
import getpass
from pinecone import Pinecone
from langchain_community.vectorstores import Pinecone as PineconeVectorStore




pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)
database = PineconeVectorStore.from_documents(
    document_list,  # 문서 리스트
    embedding,  # 임베딩 모델
    index_name=index_name,  # ✅ keyword argument로 전달해야 함!
)

In [27]:
query = '한새 비목의 설치에 대한 동의 요청이 소관 상임위원회에 회부되어 회부된 때부터 몇시간 이내에 동의 여부 를 결정하여야 합니까?'
retrived_docs = database.similarity_search(query, k=3)
retrived_docs

[Document(metadata={'source': './tax.docx'}, page_content='[전문개정 2018.4.17.]\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n제84조의2(기금운용계획안의 회부 등)\n\n\n\n\n\n① 국회는「국가재정법」제68조제1항에 따라 제출된 기금운용계획안을 회계연도 개시 30일 전까지 심의·확정한다.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n② 제1항에 따른 기금운용계획안과「국가재정법」제70조제2항에 따른 기금운용계획변경안의 회부 등에 관하여는 제84조 중 예산안 관련 규정을 준용한다.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n③ 제2항에 따라 상임위원회가 기금운용계획안 등에 대한 예비심사를 하는 경우(제84조제1항에 따라 결산에 대한 예비심사를 하는 경우를 포함한다) 기금을 운용·관리하는 부처의 소관 상임위원회와 기금사업을 수행하는 부처의 소관 상임위원회가 다를 때에는 기금을 운용·관리하는 부처의 소관 상임위원회는 기금사업을 수행하는 부처의 소관 상임위원회로부터 기금사업에 대한 의견을 들어야 한다. 다만, 기금을 운용·관리하는 부처의 소관 상임위원회의 의결일 전날까지 의견을 제시하지 아니할 경우에는 그러하지 아니하다.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n④ 제3항에 따른 기금사업을 수행하는 부처의 소관 상임위원회는 기금사업에 대한 업무보고를 들은 후 의견을 제시할 수 있다.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n[전문개정 2018.4.17.]\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n제84조의3(예산안·기금운용계획안 및 결산에 대한 공청회)\n\n\n\n\n\n예산결산특별위원회는 예산안, 기금운용계획안 및 결산에 대하여 공청회를 개최하여야 한다. 다만, 추가경정예산안, 기금운용계획변경안 또는 결산의 경우에는 위원회의 의결로 공청회를 생략할 수 있다.\n\n\n\n\n\n\n\n\n\n\n\n

In [29]:
# ------------------------------------------------------------
# 4️⃣ 프롬프트 템플릿 정의 + LLM 연결
# ------------------------------------------------------------
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4o")

rag_prompt_template = PromptTemplate(
    template="""
    You are a helpful assistant that can answer questions about Korean tax law.
    Use the following context to answer accurately and concisely.

    Context:
    {context}

    Question:
    {question}

    Answer:
    """,
    input_variables=["context", "question"]
)

# LangChain Expression Language (LCEL)로 파이프라인 구성
rag_chain = rag_prompt_template | llm

In [30]:
# ------------------------------------------------------------
# 5️⃣ 실행
# ------------------------------------------------------------
ai_message = rag_chain.invoke({
    "context": retrived_docs,   # ✅ 문자열로 전달해야 함
    "question": query
})

# 🔹 질문과 답변을 함께 출력
print("🧩 질문:")
print(query)
print("\n📚 검색된 컨텍스트:")
print(retrived_docs)
print("\n💬 답변:")
print(ai_message.content)

🧩 질문:
한새 비목의 설치에 대한 동의 요청이 소관 상임위원회에 회부되어 회부된 때부터 몇시간 이내에 동의 여부 를 결정하여야 합니까?

📚 검색된 컨텍스트:
[Document(metadata={'source': './tax.docx'}, page_content='[전문개정 2018.4.17.]\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n제84조의2(기금운용계획안의 회부 등)\n\n\n\n\n\n① 국회는「국가재정법」제68조제1항에 따라 제출된 기금운용계획안을 회계연도 개시 30일 전까지 심의·확정한다.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n② 제1항에 따른 기금운용계획안과「국가재정법」제70조제2항에 따른 기금운용계획변경안의 회부 등에 관하여는 제84조 중 예산안 관련 규정을 준용한다.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n③ 제2항에 따라 상임위원회가 기금운용계획안 등에 대한 예비심사를 하는 경우(제84조제1항에 따라 결산에 대한 예비심사를 하는 경우를 포함한다) 기금을 운용·관리하는 부처의 소관 상임위원회와 기금사업을 수행하는 부처의 소관 상임위원회가 다를 때에는 기금을 운용·관리하는 부처의 소관 상임위원회는 기금사업을 수행하는 부처의 소관 상임위원회로부터 기금사업에 대한 의견을 들어야 한다. 다만, 기금을 운용·관리하는 부처의 소관 상임위원회의 의결일 전날까지 의견을 제시하지 아니할 경우에는 그러하지 아니하다.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n④ 제3항에 따른 기금사업을 수행하는 부처의 소관 상임위원회는 기금사업에 대한 업무보고를 들은 후 의견을 제시할 수 있다.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n[전문개정 2018.4.17.]\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n제84조의3(예산안·기금운용계획안 및 결산에 대한 공청회)\n\n\n\n\n\n예산결산특별위원회는 예산안, 기금운용계획안 및 결산에 대하여 공청회를 개최